In [82]:
!pip install ortools

You should consider upgrading via the 'C:\Users\user\math_opt\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [174]:
from ortools.linear_solver import pywraplp

In [175]:
def make_data(bins_df:list, weights:list, values:list)->dict:

    bin_capacities = bins_df['base'].to_list()
    bin_minimumreq = list()
    # flagに従って、bin_capacitiesを上乗せする
    # bin_capacities
    # flagに従って、bin_capacitiesの最小値を設定する
    # bin_minimumreq

    for i, flg in enumerate(bins_df.loc[:,"flag"]):
        if flg == 'm':
            bin_capacities[i] = bin_capacities[i] + 10000
            bin_minimumreq.append(0)
        elif flg == 'i':
            if bin_capacities[i] > 500000:
                bin_minimumreq.append(bin_capacities[i] - 50000)
                bin_capacities[i] = bin_capacities[i] + 50000
            else:
                bin_minimumreq.append(bin_capacities[i] - 10000)
                bin_capacities[i] = bin_capacities[i] + 10000
        else:
            raise ValueError("フラグの値が不正です")

    data = {}
    data['weights'] = weights
    data['values'] = values
    assert len(data['weights']) == len(data['values'])
    data['num_items'] = len(data['weights'])
    data['all_items'] = range(data['num_items'])

    data['bin_capacities'] = bin_capacities
    data['num_bins'] = len(data['bin_capacities'])
    data['all_bins'] = range(data['num_bins'])
    return data, bin_minimumreq

In [180]:
def slv_multiknap(data,bin_minimumreq):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if solver is None:
        print('SCIP solver unavailable.')
    # x[i, b] = 1 if item i is packed in bin b.
    x = {}
    for i in data['all_items']:
        for b in data['all_bins']:
            x[i, b] = solver.BoolVar(f'x_{i}_{b}')

    # それぞれのアイテムはバッグに1つ以上入らない
    for i in data['all_items']:
        solver.Add(sum(x[i, b] for b in data['all_bins']) <= 1)

    # キャパシティの上限・下限の設定
    for b in data['all_bins']:
        # 上限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) <= data['bin_capacities'][b])
        # 下限容量
        solver.Add(
            sum(x[i, b] * data['weights'][i]
                for i in data['all_items']) >= bin_minimumreq[b])

    # 個数の最大化を目標にする
    objective = solver.Objective()
    for i in data['all_items']:
        for b in data['all_bins']:
            objective.SetCoefficient(x[i, b], data['values'][i])
    objective.SetMaximization()

    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print(f'最大個数: {objective.Value()}\n')
        txt = f'最大個数: {objective.Value()}\n'
        total_weight = 0
        for b in data['all_bins']:
            print(f'Bin {b}')
            txt += f'Bin {b}'
            bin_weight = 0
            bin_value = 0
            for i in data['all_items']:
                if x[i, b].solution_value() > 0:
                    print(
                        f"Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    )
                    txt += f"Item {i} 重量: {data['weights'][i]} 個数: {data['values'][i]}"
                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
            print(f'パック内の総重量は: {bin_weight}')
            txt += f'パック内の総重量は: {bin_weight}'
            print(f'パック内の個数は: {bin_value}\n')
            txt += f'パック内の個数は: {bin_value}\n'
            total_weight += bin_weight
        print(f'総重量は: {total_weight}')
        txt += f'総重量は: {total_weight}'
        with open('result.txt', 'w') as f:
            f.write(txt)
        return True
    else:
        # print('最適解なし')
        return False

In [181]:
import pandas as pd
bins_df_original = pd.read_csv('class.csv')
items_df = pd.read_csv('teg.csv')
weights = items_df['list'].to_list()
values = [ 1 for x in weights]

In [182]:
#classの組み合わせ作成
import itertools

bins_max = len(bins_df_original)
res = False
for i in range(0,bins_max):
    bins_combs = itertools.combinations(bins_df_original.index, len(bins_df_original) - i)
    for bins_comb in bins_combs:
        bins_df = bins_df_original.iloc[list(bins_comb)]
        data, bin_minimumreq = make_data(bins_df,weights,values)
        res = slv_multiknap(data, bin_minimumreq)
        if res:
            break
    if res:
        break
print(f'{i+1}回目で処理完了')

最大個数: 34.99999999999999

Bin 0
Item 2 重量: 141386 個数: 1
Item 4 重量: 132737 個数: 1
Item 34 重量: 432322 個数: 1
パック内の総重量は: 706445
パック内の個数は: 3

Bin 1
Item 5 重量: 190000 個数: 1
Item 21 重量: 359263 個数: 1
Item 45 重量: 148511 個数: 1
パック内の総重量は: 697774
パック内の個数は: 3

Bin 2
Item 1 重量: 440550 個数: 1
Item 11 重量: 503492 個数: 1
Item 22 重量: 400169 個数: 1
Item 35 重量: 813000 個数: 1
Item 37 重量: 206853 個数: 1
パック内の総重量は: 2364064
パック内の個数は: 5

Bin 3
Item 32 重量: 236830 個数: 1
Item 43 重量: 583000 個数: 1
パック内の総重量は: 819830
パック内の個数は: 2

Bin 4
Item 9 重量: 917741 個数: 1
Item 14 重量: 169378 個数: 1
Item 20 重量: 372900 個数: 1
Item 30 重量: 308000 個数: 1
パック内の総重量は: 1768019
パック内の個数は: 4

Bin 5
Item 24 重量: 335610 個数: 1
Item 31 重量: 173830 個数: 1
Item 40 重量: 646338 個数: 1
パック内の総重量は: 1155778
パック内の個数は: 3

Bin 6
Item 7 重量: 122100 個数: 1
Item 18 重量: 671031 個数: 1
パック内の総重量は: 793131
パック内の個数は: 2

Bin 7
Item 3 重量: 519970 個数: 1
Item 6 重量: 388245 個数: 1
Item 8 重量: 456236 個数: 1
Item 10 重量: 747604 個数: 1
Item 26 重量: 220000 個数: 1
Item 28 重量: 619520 個数: 1
Item 29 重量: 4500